In [1]:
import sympy
import ipywidgets
import numpy as np
import matplotlib.pyplot as plt
import nashpy

In [2]:

class eq():
    def __init__(self):
        points = 101
        self.x,step = np.linspace(0,1,points,retstep=True)        
        
        
        #--- Symbolic symbols used in expected utility functions
        # n, PR,C_r, D, p_a, B, pi, C_v, p_v, C_e, C_d, p_e
        self.n, self.PR, self.C_r, self.D, self.r = sympy.symbols('n PR C_r D r', real=True)
        self.p_a, self.B, self.pi, self.C_v,  = sympy.symbols('p_a B pi C_v', real=True)
        self.C_e, self.C_d = sympy.symbols('C_e C_d', real=True)
        
        self.se = sympy.symbols('sigma_e', real=True)
        self.sv = sympy.symbols('sigma_v', real=True)
        
        
    def findNashEq(self, n, PR, p_a, B, pi, C_v, C_r, C_e, C_d):
        '''This function solves for the mixed strategy of the JC and RP. 
        sigma_e : the probability that the RP will execute the job
        1-sigma_e: the probability that the RP will not execute the job
        sigma_v: the probability that the JC will verify the result
        1-sigma_v: the probability that the JC will not verify the result'''
#         n, PR,C_r, D, p_a, B, pi, C_v, p_v, C_e, C_d, p_e
        
        
        
        
        D = pi*(PR + n)
        
        # J1 and J3, JC verifies
        # J2 and J4, JC does not verify
        J1,J2,J3,J4 = self.JCU(n, D, p_a, B, pi, C_v, C_r)
        
        # Ev: JC's expected value for verifying the result
        Ev = sigma_e*J1 + (1-sigma_e)*J3
        print("E[v]")
        display(Ev)
        # Ep: JC's expected value for not verifying the result
        Ep = sigma_e*J2 + (1-sigma_e)*J4 
        # RP solves for sigma_e such that the JC is ambivalent between verifying and not verifying by setting the two expected values equal to each other
        nse = sympy.solve([Ev - Ep], [sigma_e])[sigma_e]
        
        # Substitute nse(nash equlibrium sigma_e) back in to JC's expected utility function
        print("E[v] with nse")
        Evne = nse*J1 + (1-nse)*J3
        display(Evne.expand().simplify())
        
        # R1 and R2, RP executes 
        # R3 and R4, RP does not execute
        R1,R2,R3,R4 = self.RPU(n, D, p_a, pi, C_e, C_d)
        # Ee: RP's expected value for executing the job
        Ee = sigma_v*R1 + (1-sigma_v)*R2
        print("E[e]")
        display(Ee)
        # Ed: RP's expected value for sending a deceiving result
        Ed = sigma_v*R3 + (1-sigma_v)*R4
        # JC solve sor sigma_v such that the RP is ambivalent between executing and deceiving 
        nsv = sympy.solve([Ee - Ed], [sigma_v])[sigma_v]
        
        
        # Verify that when substituting nse back in the JC expected values are equal.
        check_e = nse*J1 + (1-nse)*J3 - ( nse*J2 + (1-nse)*J4 )
        print("e is equilibrium: {}, check is {}".format(check_e.simplify()==sympy.numbers.Zero, check_e.simplify()))
        
        
        return nse, nsv
        
    def JCU(self, n, D, r, p_a, B, pi, C_v, C_r):
        '''The JC's utility functions'''
        J1 = p_a*(B-pi-C_v) + (1-p_a)*( (p_a**n)*(B+r-C_v) + (1-p_a**n)*(B-D-C_r-C_v))
        J2 = B - pi
        J3 = (p_a**n)*(r-C_v) + (1-p_a**n)*(-C_r-C_v-D)
        J4 = -pi
        
        return J1, J2, J3, J4
    
#     def RPU(self, p_a, pi, C_e, C_d, n, D):
    def RPU(self, n, D, r, p_a, pi, C_e, C_d):
        '''The RP's utility functions'''
        R0 = p_a*(pi-C_e) + (1-p_a)*( (p_a**n)*(-C_e-D) + (1-p_a**n)*(r-C_e))
        R1 = pi - C_e
        R2 = (p_a**n)*(-C_d-D) + (1-p_a**n)*(r-C_d)
        R3 = pi-C_d
        
        return R0, R1, R2, R3
    
        

In [3]:
e = eq()

In [20]:

D = e.pi*(e.PR + e.n)
r = e.pi

RU = e.RPU(e.n, D, r, e.p_a, e.pi, e.C_e, e.C_d)
# for ix, u in enumerate(RU):
#     print("RU: {}".format(ix))
#     display(u)
    
# left
# display(RU[0].expand())
# display(RU[2].expand())

# shared = -e.C_e  -e.D*e.p_a**e.n - e.p_a**e.n*e.r + e.r
# shared = -e.C_e  -e.PR*e.p_a**e.n*e.pi - e.n*e.p_a**e.n*e.pi - e.p_a**e.n*e.r + e.r
shared = -e.C_e  -e.PR*e.p_a**e.n*e.pi - e.n*e.p_a**e.n*e.pi - e.p_a**e.n*e.pi + e.pi

display((RU[0].expand() - shared).collect(e.p_a).collect(e.p_a**e.n).collect(e.pi))
display(RU[2].expand() - shared)


# right
display(RU[1].expand())
display(RU[3].expand()) # *

p_a*p_a**n*pi*(PR + n + 1)

-C_d + C_e

-C_e + pi

-C_d + pi

In [5]:
JU = e.JCU(e.n, e.D, e.r, e.p_a, e.B, e.pi, e.C_v, e.C_r)
for ix, u in enumerate(JU):
    print("JU: {}".format(ix))
    display(u)

JU: 0


p_a*(B - C_v - pi) + (1 - p_a)*(p_a**n*(B - C_v + r) + (1 - p_a**n)*(B - C_r - C_v - D))

JU: 1


B - pi

JU: 2


p_a**n*(-C_v + r) + (1 - p_a**n)*(-C_r - C_v - D)

JU: 3


-pi